# Simulated Annealing for Mars Lander

In [1]:
import numpy as np
import re, math, random, pdb, json 

In [2]:
import peforth
peforth.bps = [i for i in range(100)]  # 預設有這麼多 Breakpoint ID
def bp(id=None,locals=None):
    # Breakpoint ID 不能超過 peforth.bps 保留，超過的無效。
    if id==None: 
        id = 0
        prompt='bp> '
    else:
        prompt="{}>".format(id)
    if id in peforth.bps: peforth.push(locals).ok(prompt, cmd="to _locals_")
peforth.bp = bp
peforth.push(bp).dictate("py: vm.bp=pop()")  # [ ] 忘了 vm.bp 有何用途？
peforth.dictate('''
    \ ------ breakpoint ------------------------------------------------------------
    : bp ." Usage: peforth.bp(11,locals())  # drop a breakpoint with ID=11" cr ;
    : bl // ( -- ) List all breakpoints
        __main__ :> peforth.bps 
        <py>
        bps = pop()
        print('Disabled breakpoints:')
        for i in range(len(bps)):
            if not bps[i]: 
                print(i, end=' ')
        print(); print('Enabled breakpoints:')
        count = 0
        for i in range(len(bps)):
            if bps[i]: 
                print(i, end=' ')
                count += 1
        print(); print('Enabled breakpoints count: {}/{}'.format(count,len(bps)))
        </py> cr ;
        /// Breakpoint commands:
        ///	  bl  - list all breakpoints (capital BL is white space) 
        ///	  be  - enable breakpoints, e.g. be 1 2 3 
        ///	  bd  - disable breakpoints, e.g. bd 1 2 3 
        ///	  be* - enable all breakpoints
        ///	  bd* - disable all breakpoints 
        /// Also: 
        ///   for i in [11,22,33]: peforth.bps[i]=0	 # disable breakpoints 11,22,33 
        ///   for i in [11,22,33]: peforth.bps[i]=i	 # enable  breakpoints 11,22,33 
    
    : bd // ( <1 2 3 4...> -- ) Disable listed breakpoints 
        CR word ( line ) __main__ :> peforth.bps ( line bps )
        <py>
        line, bps = pop(1), pop(0)
        points = map(int, line.split(' '))
        for i in points: bps[i] = 0
        </py> ; 
        ' bl :> comment last :: comment=pop(1)
    
    : be // ( <1 2 3 4...> -- ) Enable listed breakpoints 
        CR word ( line ) __main__ :> peforth.bps ( line bps ) 
        <py>
        line, bps = pop(1), pop(0)
        points = map(int, line.split(' '))
        for i in points: bps[i] = i
        </py> ; 
        ' bl :> comment last :: comment=pop(1)
    
    : bd* // ( -- ) Disable all breakpoints 
        __main__ :> peforth.bps	 ( bps ) 
        <py>
        bps = pop()
        for i in range(len(bps)): bps[i] = 0
        </py> ;
        ' bl :> comment last :: comment=pop(1)
    
    : be* // ( -- ) Enable all breakpoints 
        __main__ :> peforth.bps	 ( bps ) 
        <py>
        bps = pop()
        for i in range(len(bps)): bps[i] = i
        </py> ;
        ' bl :> comment last :: comment=pop(1)
    \ ------ xtack ------------------------------------------------------------
    [] value xstack xstack py: vm.xstack=pop() // ( -- array ) The xstack. 掛進 vm 就可以直接以 py> xstack 取用。
    : x@ xstack :> [-1] ; // ( -- n ) Get TOS of the xstack
    : x> xstack :> pop() ; // ( -- n ) Pop the xstack
    : >x xstack :: append(pop()) ; // ( n -- ) Push n into the xstack
    : .sx xstack . ; // ( -- ) List xstack 
    : xdrop x> drop ; // ( X: ... a -- X: ... ) drop xstack 
    : xdropall [] to xstack ; // ( X: ... -- X: empty ) clear xstack 
    ''')
%f version ==>


reDef unknown
reDef -->
p e f o r t h    v1.24
source code http://github.com/hcchengithub/peforth
Type 'peforth.ok()' to enter forth interpreter, 'exit' to come back.

version ==>
 1.24 (<class 'str'>)


In [20]:
gravity = -3.711        # constant on Mars
g = gravity             # 到了二維以上，重力需轉成向量

class State:

    def __init__(self,step=0,x=0,y=0,hspeed=0,
            vspeed=0,fuel=0,angle=0,power=0,force=0):
        self.step   = step
        self.x      = x
        self.y      = y
        self.hspeed = hspeed
        self.vspeed = vspeed
        self.fuel   = fuel
        self.angle  = angle
        self.power  = power
        self.force  = force

    def next(self,angle,power):  
        # input angle,power 都是 offset 分別屬 -15~15 以及 -1,0,+1 
        self.step   += 1
        self.angle  += angle         # 記錄這步的 command 
        self.power  += power         # 記錄這步的 command 
        self.fuel   -= self.power
        self.x       = self.x       # episode-1 不變
        self.hspeed  = self.hspeed  # episode-1 不變
        self.force   = g + self.power
        self.y       = self.y + self.vspeed + (1/2)*self.force
        self.vspeed += self.force
        assert power in [-1, 0, 1], 'expected input power is an offset [-1, 0, +1], given {}'.format(power)
        assert angle in range(-15,16), 'input angle range [-15:16], given {}'.format(angle)
        assert self.power in range(5), "power value {} out of range".format(self.power)
        assert self.angle in range(-90,91), "angle value {} out of range".format(self.angle)
        return self  # 傳回值只是方便測試，可以這樣寫： %f state :> next(0,3.711) -->

    def __str__(self):  
        return str(self.__dict__)

# 測試兼範例
# 模擬自由落體
state = State(0,0,2500,0,0,500,0,0,0)
%f 36 [for] state :> next(0,0) [next] 
%f ( Free fall 墜毀 at step 36 ) state --> cr 
# 模擬安全降落
state = State(0,0,2500,0,0,500,0,0,0)
%f 9 [for] state :> next(0,0) [next] 
%f 4 [for] state :> next(0,1) [next] 
%f 91 13 - [for] state :> next(0,0) [next] 
%f state --> cr 
# 模擬飛向太空
state = State(0,0,2500,0,0,500,0,0,0)
%f 4 [for] state :> next(0,1) [next] 
%f 83 4 - [for] state :> next(0,0) [next] 
%f state -->

( Free fall 墜毀 at step 36 ) state --> {'step': 36, 'x': 0, 'y': 95.2720000000003, 'hspeed': 0, 'vspeed': -133.59599999999998, 'fuel': 500, 'angle': 0, 'power': 0, 'force': -3.711} (<class '__main__.State'>)

state --> {'step': 91, 'x': 0, 'y': 97.6045000000063, 'hspeed': 0, 'vspeed': -15.700999999999876, 'fuel': 178, 'angle': 0, 'power': 4, 'force': 0.28900000000000015} (<class '__main__.State'>)

state --> {'step': 83, 'x': 0, 'y': 3004.4604999999906, 'hspeed': 0, 'vspeed': 17.987000000000002, 'fuel': 174, 'angle': 0, 'power': 4, 'force': 0.28900000000000015} (<class '__main__.State'>)


In [2]:
# 改寫 for Mars Lander 乍看以為要頭痛一會兒，其實 Mars Lander 的 genome 很簡單，就是一串 (int,int) 
# pairs 而已。前面基礎討論有提到，以 SA 而言就是 mutate 某一個 gene 也就是 command pair 而已。但是考慮 Mars
# Lander 的限制條件，以 epidode-1 而言，command 的改變要做的決定是 -1 0 +1 其中之一才對，而非 0~4. 

# 長度的問題，
# 我想的是需要另一個 function 不夠時直接 copy 最後一個 gene 頂替即可。
# 會發現不夠的是 call fitness function 的人，那就是由它處理。

def get_random_neighbour(state):
    
    neighbour = state[:]  # Deep copy 利用 [:] slice copy, 若用 list(state) 不行！

    idx = random.randint(0,len(state)-1) # 選 1 個 gene 
    cmd = random.randint(-1, 1)  # -1, 0, +1 當中選 1 個 command 

    neighbour[idx][1] = cmd
    
    return neighbour

In [3]:
# ○ 成功著陸的，油料剩越多的越好。
# ○ 一個油料耗盡一個墜毀哪個好？墜毀時的速度減到+-40 以內需要多少油料？
# ○ 油料耗盡時的高度距離與速度以自由落體觸地，然後把速度調整到+-40 以內需要多少油料？
# ○ 由以上三點分析看來， cost_of_state(state) where 
#       state = ( step, x, [y], hspeed, [vspeed], [fuel], rotate, power ) 只參考 state 裡的三項 data. 
# ○ 根據我之前對 y=100 的研究 該 state 來自 log 的最後一筆 state. Simulation 時，該 state 就是 y 首次跨
#   越 flat ground 高度的該 step, 也就是觸地著陸之時. 這個 cost_of_state() 不用管，它只管算出分數即可。
# ○ cost 以 0 為目標。Score 用油料剩餘量計算，由 fuel0 扣下來可到負值，推估範圍是：[-fuel : fuel0]。換算
#   成 cost 就是 cost = fuel0 - score. 核算，當 score 為 fuel0 時 cost 為 0 無誤； 當 score 為 -fuel0
#   時 cost 為 2fuel0 -- 好高，也無誤。 直接用這個數字，配合 SA 的 Temperature 設定要用 visualized 
#   graphs 來訂定 即可。

def cost_of_state(state):
    state = State(**state_all_power.__dict__)  #
    score = None  # 有可能是負的，先給個看得出異常的初值。

    # 先判斷是否觸地。若觸地進一步查看速度合格是否合格。若未觸地就是飛走了或油料耗盡。
    if state.y <= 100:
        # 觸地了，進一步查看速度是否合格
        if state.vspeed >= -40 : # 剛從空中下來觸地， vspeed 一定是負值；墜毀的一定比 -40 小
            # 安全降落
            score = state.fuel
        else:  # 墜毀
            # 模擬噴火把速度降到合格範圍內。所剩油料就是成績。
            while True:
                state.next(0,min(state.power + 1, 4))  # 有個調整的過程，不是直接填 4 ！
                if state.vspeed >= -40:
                    score = state.fuel
                    break
    else:  # 還在空中，飛走了或油料耗盡。
        # 先模擬 free fall 觸地。
        while True: 
            state.next(0,max(state.power-1,0))  # 半空中開始模擬 free fall 小心別又錯了，不是直接填 0 ！！
            if state.y <= 100: break
        # 然後噴火把速度降到合格範圍內，所剩油料就是成績。
        while True:
            if state.vspeed >= -40: # 剛從空中下來觸地， vspeed 一定是負值；墜毀的一定比 -40 小。
                # 達標了
                score = state.fuel
                break
            else: # 未達標，繼續噴火把負的 vspeed 速度降到合格範圍內
                state.next(0,min(state.power + 1, 4))

    %f score -->

    
    
    cost = fuel0  # 初始油料
    
    return lost - sum(clue)


In [97]:
state = State(**state_all_power.__dict__)  #
score = None  # 有可能是負的，先給個看得出異常的初值。

# 先判斷是否觸地。若觸地進一步查看速度合格是否合格。若未觸地就是飛走了或油料耗盡。
if state.y <= 100:
    # 觸地了，進一步查看速度是否合格
    if state.vspeed >= -40 : # 剛從空中下來觸地， vspeed 一定是負值；墜毀的一定比 -40 小
        # 安全降落
        score = state.fuel
    else:  # 墜毀
        # 模擬噴火把速度降到合格範圍內。所剩油料就是成績。
        while True:
            state.next(0,min(state.power + 1, 4))  # 有個調整的過程，不是直接填 4 ！
            if state.vspeed >= -40:
                score = state.fuel
                break
else:  # 還在空中，飛走了或油料耗盡。
    # 先模擬 free fall 觸地。
    while True: 
        state.next(0,max(state.power-1,0))  # 半空中開始模擬 free fall 小心別又錯了，不是直接填 0 ！！
        if state.y <= 100: break
    # 然後噴火把速度降到合格範圍內，所剩油料就是成績。
    while True:
        if state.vspeed >= -40: # 剛從空中下來觸地， vspeed 一定是負值；墜毀的一定比 -40 小。
            # 達標了
            score = state.fuel
            break
        else: # 未達標，繼續噴火把負的 vspeed 速度降到合格範圍內
            state.next(0,min(state.power + 1, 4))
    
%f score -->



11>

 state -->


state --> {'step': 130, 'x': 2500, 'y': 26.049999999989105, 'hspeed': 0, 'vspeed': -150.43000000000004, 'fuel': 168, 'angle': 0, 'power': 0, 'force': -3.711} (<class '__main__.State'>)
11>

 exit


11>22>

...

22>22>

 state -->


state --> {'step': 533, 'x': 2500, 'y': -39540.139500000354, 'hspeed': 0, 'vspeed': -39.96300000000083, 'fuel': -1438, 'angle': 0, 'power': 4, 'force': 0.28900000000000015} (<class '__main__.State'>)
22>

 bd* exit


22>score --> -1438 (<class 'int'>)


In [4]:
# fitness function 
# https://en.wikipedia.org/wiki/Hyperparameter_(machine_learning)
def sa(initial):
    current = initial
    current_cost = cost_of_state(current)
    temp = 1.0  # <---- [ ] Hyperparameter
    num_iterations = 0

    while current_cost > 0:
        neighbour = get_random_neighbour(current)
        neighbour_cost = cost_of_state(neighbour)

        cost_delta = neighbour_cost - current_cost

        if cost_delta <= 0 or random.random() < math.exp(-cost_delta/temp):
            current, current_cost = neighbour, neighbour_cost

        num_iterations += 1
        if num_iterations % 500 == 0 and temp > 0.20:  # <---- [ ] Hyperparameter
            temp -= 0.05  # <---- [ ] Hyperparameter

    return current, num_iterations

In [5]:
def epoch(state=None):
    nationalities = [ 'dane',      'brit',   'swede',       'norwegian', 'german'    ]
    colours       = [ 'yellow',    'red',    'white',       'green',     'blue'      ]
    animals       = [ 'horse',     'cat',    'bird',        'fish',      'dog'       ]
    beverages     = [ 'water',     'tea',    'milk',        'coffee',    'root beer' ]
    cigars        = [ 'pall mall', 'prince', 'blue master', 'dunhill',   'blends'    ]

    random.shuffle(nationalities)
    random.shuffle(colours      )
    random.shuffle(animals      )
    random.shuffle(beverages    )
    random.shuffle(cigars       )
    
    attributes = [colours, nationalities, beverages, cigars, animals]

    NUM_HOUSES = 5
    initial = []
    s = '['
    
    if state: 
        initial = state; 
    else:
        for i in range(NUM_HOUSES):
            initial.append([attr[i] for attr in attributes])
    
    
    # random.seed(100)

    solution, iterations = sa(initial)

    for house in solution:
        s += str(house) + ',';
    s = s[:-1] + ']';
    return s,iterations

In [6]:
if __name__ == "__main__":
    ss = '';
    for i in range(10000):
        s, iter = epoch()
        ss += "{} Number of iterations:{}\n".format(s, iter)
        print('.', end="")
    %f ss char results.txt writeTextFile 
    %f .( Done !!! ) cr

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

http://localhost:8888/lab/workspaces/auto-m/tree/OneDrive/%E6%96%87%E4%BB%B6/Jupyter%20Notebooks/I%20study%20Pandas.ipynb

In [7]:
import pandas as pd

## 研究 pandas read data 的眾多方法

我看其中就是 pd.read_clipboard() 好用。先把資料抓進 clipboard 然後執行之即得。
研究 iterations 數的分布的目的是想藉以調整 Annealing Algorithm 的 hyper-parameter 

In [8]:
data = pd.read_clipboard()
data

,Data
0,8910
1,9532
2,6423
3,6776
4,22155
...,...
995,13802
996,6385
997,7525
998,8103


In [41]:
%f data :> describe() ==>
%f data :> Data.median() --> 

data :> describe() ==>
                 Data
count    1000.000000
mean    13933.352000
std     13448.507494
min      1242.000000
25%      6925.250000
50%      8452.000000
75%     15068.000000
max    113487.000000 (<class 'pandas.core.frame.DataFrame'>)
data :> Data.median() --> 8452.0 (<class 'numpy.float64'>)


In [45]:
data = pd.read_clipboard()
data

,Iterations
0,4851
1,6097
2,20168
3,10302
4,27273
...,...
9995,6986
9996,9056
9997,18983
9998,7096


In [47]:
%f data :> describe() ==>
%f data :> Iterations.median() --> 

data :> describe() ==>
          Iterations
count   10000.00000
mean    13178.39830
std     12065.95423
min        45.00000
25%      6995.75000
50%      8426.00000
75%     14400.25000
max    152908.00000 (<class 'pandas.core.frame.DataFrame'>)
data :> Iterations.median() --> 8426.0 (<class 'numpy.float64'>)
